In [1]:
from DataLoader import getdata

In [2]:
Train, Test = getdata()

Going through data, please be patient perhaps go make some tea it will take 5-10 mins to load up
## Sanity check


IndexError: too many indices for array

In [ ]:
print(Train.shape)
print(Test.shape)
print(Train[0,0])

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

# Generate dummy data
import numpy as np
y_train = Train[:,-1]
x_train = Train[:,1:-1]
Train[np.where(Train[:,-1]==-1),-1] = 10

y_test = Test[:,-1]
x_test = Test[:,1:-1]
Test[np.where(Test[:,-1]==-1),-1] = 10

model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, activation='relu', input_dim=Train.shape[1]-2))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(11, activation='softmax'))

sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=20,
          batch_size=128)
score = model.evaluate(x_test, y_test, batch_size=128)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import LSTM
# Generate dummy data
import numpy as np
y_train = Train[:,-1]
x_train = Train[:,1:-1]
Train[np.where(Train[:,-1]==-1),-1] = 10

y_test = Test[:,-1]
x_test = Test[:,1:-1]
Test[np.where(Test[:,-1]==-1),-1] = 10


model = Sequential()
model.add(LSTM(64,input_shape=[Train.shape[1]-2,1],recurrent_dropout=0.5))

model.add(Dense(11, activation='sigmoid'))
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
print(x_train.shape,y_train.shape)
model.fit(np.reshape(x_train,[48750,3696,1]), y_train, batch_size=64, epochs=2)
score = model.evaluate(x_test, y_test, batch_size=64)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
import numpy as np
y_train = Train[:,-1]
x_train = Train[:,1:-1]
Train[np.where(Train[:,-1]==-1),-1] = 10

y_test = Test[:,-1]
x_test = Test[:,1:-1]
Test[np.where(Test[:,-1]==-1),-1] = 10

model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=[Train.shape[1]-2,1]))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(11, activation='sigmoid'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(np.reshape(x_train,[x_train.shape[0],x_train.shape[1],1]), y_train, batch_size=16, epochs=10)
score = model.evaluate(x_test, y_test, batch_size=16)